# problem


create data set randomly

In [1]:
import numpy as np
import pandas as pd

np.random.seed(42)

num_customers = 20

all_coords = np.random.rand(num_customers, 2) * 100

# depot_coords = np.array([[50, 50]])

# all_coords = np.vstack([customer_coords])

coords_df = pd.DataFrame(all_coords, columns=['x', 'y'])

coords_df.to_csv('vrp_data.csv', index=False)

print({
        "customers": num_customers,
        "x": coords_df['x'].tolist(),
        "y": coords_df['y'].tolist()
    }
)
# print(all_coords)
print(coords_df)

{'customers': 20, 'x': [37.454011884736246, 73.1993941811405, 15.601864044243651, 5.8083612168199465, 60.11150117432088, 2.0584494295802447, 83.24426408004217, 18.182496720710063, 30.42422429595377, 43.194501864211574, 61.18528947223795, 29.214464853521815, 45.606998421703594, 19.967378215835975, 59.24145688620425, 60.75448519014384, 6.505159298527952, 96.56320330745594, 30.46137691733707, 68.42330265121569], 'y': [95.07143064099162, 59.86584841970366, 15.599452033620265, 86.61761457749351, 70.80725777960456, 96.99098521619943, 21.233911067827616, 18.34045098534338, 52.475643163223786, 29.122914019804192, 13.949386065204184, 36.63618432936917, 78.51759613930136, 51.42344384136116, 4.645041271999773, 17.052412368729154, 94.88855372533332, 80.83973481164611, 9.767211400638388, 44.01524937396013]}
            x          y
0   37.454012  95.071431
1   73.199394  59.865848
2   15.601864  15.599452
3    5.808361  86.617615
4   60.111501  70.807258
5    2.058449  96.990985
6   83.244264  21.2

# encoding

In [2]:
import numpy as np
import pandas as pd

coords_df = pd.read_csv('vrp_data.csv')

all_coords = coords_df.values
print(all_coords)
# customer_indices = np.arange(0, len(all_coords))

# customer_indices

[[37.45401188 95.07143064]
 [73.19939418 59.86584842]
 [15.60186404 15.59945203]
 [ 5.80836122 86.61761458]
 [60.11150117 70.80725778]
 [ 2.05844943 96.99098522]
 [83.24426408 21.23391107]
 [18.18249672 18.34045099]
 [30.4242243  52.47564316]
 [43.19450186 29.12291402]
 [61.18528947 13.94938607]
 [29.21446485 36.63618433]
 [45.60699842 78.51759614]
 [19.96737822 51.42344384]
 [59.24145689  4.64504127]
 [60.75448519 17.05241237]
 [ 6.5051593  94.88855373]
 [96.56320331 80.83973481]
 [30.46137692  9.7672114 ]
 [68.42330265 44.01524937]]


In [3]:
import plotly.express as px

fig = px.scatter(coords_df, x='x', y='y', color=['Depot'] + ['Customer']*20, 
                 labels={'color': 'Location Type'}, title='Customer and Depot Locations')

fig.show()


ValueError: All arguments should have the same length. The length of argument `color` is 21, whereas the length of  previously-processed arguments ['x', 'y'] is 20

# population intilaztion

In [ ]:
def initialize_population(pop_size, num_customers):
    customer_indices = np.arange(1, num_customers + 1)  # Exclude depot
    population = []
    for _ in range(pop_size):
        route = np.random.permutation(customer_indices)  # Random customer order
        route = np.concatenate(([0], route, [0]))       # Add depot at start and end
        population.append(route)

    return np.array(population)


# huristic function

In [ ]:
def calculate_distance(route, coords):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += np.linalg.norm(coords[route[i]] - coords[route[i + 1]])
    return total_distance


# selection function

In [ ]:
import random

def selection(population, fitness_scores, num_parents):
    sorted_indices = np.argsort(fitness_scores)
    ranked_population = [population[i] for i in sorted_indices]
    selected_parents = ranked_population[:num_parents]
    return selected_parents

# crossover function

In [ ]:
def crossover(parent1, parent2):
    size = len(parent1) - 2  

    point = np.random.randint(1, size)  

    child = [-1] * len(parent1) 
    child[0], child[-1] = 0, 0  
    child[1:point + 1] = parent1[1:point + 1]  
    pointer = point + 1
    
    for gene in parent2:
        if gene not in child:
            child[pointer] = gene
            pointer += 1
    return np.array(child)


In [ ]:
def mutate(route, mutation_rate):
    if np.random.rand() < mutation_rate:
        idx1, idx2 = np.random.choice(range(1, len(route) - 1), 2, replace=False)
        route[idx1], route[idx2] = route[idx2], route[idx1]
    return route


<h1>evlolve</h1>
<p>
    <!-- <img src="spinning-cat.gif" alt="Spinning Cat"> -->
</p>

<style>
    h1 {
        color: red;
    }
    p {
        position: relative;
        /* animation: moveRightLeft 2s infinite ease-in-out; */
    }
    img
    {
        width :100px;
        height :100px;
    }

    @keyframes moveRightLeft {
        0% {
            left: 0;
        }
        50% {
            left: 100px;
        }
        100% {
            left: 0;
        }
    }
</style>

In [ ]:
def validate_and_repair(route, num_customers):
    all_customers = set(range(1, num_customers + 1))
    visited = set(route[1:-1])  # Exclude depot
    missing_customers = list(all_customers - visited)
    duplicates = [c for c in route[1:-1] if route[1:-1].tolist().count(c) > 1]

    for i in range(1, len(route) - 1):
        if route[i] in duplicates:
            route[i] = missing_customers.pop()
            duplicates.remove(route[i])
    return route


In [ ]:
def evolve_population(population, coords, num_customers, mutation_rate, retain_rate):
    fitness_scores = [1 / calculate_distance(route, coords) for route in population]

    sorted_indices = np.argsort(fitness_scores)[::-1]
    retain_length = int(len(population) * retain_rate)
    parents = [population[i] for i in sorted_indices[:retain_length]]

    children = []
    while len(children) < len(population) - len(parents):

        p1, p2 = parents[0], parents[1]  # Just use the first two parents

        child = crossover(p1, p2)
        children.append(child)


    next_gen = parents + children
    next_gen = [validate_and_repair(mutate(route, mutation_rate), num_customers) for route in next_gen]

    return np.array(next_gen)


# trying the alogrithm 

In [ ]:
import numpy as np

def run_ga(coords, num_customers, population_size=100, generations=1000, mutation_rate=0.02, retain_rate=0.2):
    
    population = initialize_population(population_size, num_customers)

    best_route = None
    best_distance = float('inf')
    best_distance_arr = []
    gen_of_best_distance = []

    for gen in range(generations):
        population = evolve_population(population, coords, num_customers, mutation_rate, retain_rate)

        distances = [calculate_distance(route, coords) for route in population]
        min_distance_idx = np.argmin(distances)
        min_distance = distances[min_distance_idx]

        # Update best solution if found
        if min_distance < best_distance:
            best_distance = min_distance
            best_route = population[min_distance_idx]
            print(f"Generation {gen + 1}/{generations}: Best Distance = {best_distance:.2f}")
            best_distance_arr.append(float(best_distance))
            gen_of_best_distance.append(gen + 1)

        # if gen % 50 == 0 or gen == generations - 1:
        #     print(f"Generation {gen + 1}/{generations}: Best Distance = {best_distance:.2f}")

    return {
        "best route": best_route.tolist(),
        "best distance": float(best_distance),
        "best distance array": best_distance_arr,
        "generations of best distance": gen_of_best_distance
    }



In [ ]:
# coords = coords_df[['x', 'y']].to_numpy()
# print(coords)
num_customers = len(all_coords) - 1

output = run_ga(all_coords, num_customers)
print(output)
# print(gen_of_best_distance)
# print("maybe a best route:", best_route)
# print("a maybe with a qeustion mark best distance  :", best_distance)


Generation 1/1000: Best Distance = 838.06
Generation 4/1000: Best Distance = 823.06
Generation 9/1000: Best Distance = 820.90
Generation 10/1000: Best Distance = 740.46
Generation 16/1000: Best Distance = 735.78
Generation 18/1000: Best Distance = 734.82
Generation 30/1000: Best Distance = 659.57
Generation 31/1000: Best Distance = 631.79
Generation 34/1000: Best Distance = 592.07
Generation 44/1000: Best Distance = 585.99
Generation 47/1000: Best Distance = 585.57
Generation 67/1000: Best Distance = 582.33
Generation 68/1000: Best Distance = 580.70
Generation 162/1000: Best Distance = 580.49
Generation 178/1000: Best Distance = 568.20
Generation 213/1000: Best Distance = 559.16
Generation 219/1000: Best Distance = 555.13
Generation 228/1000: Best Distance = 553.50
Generation 236/1000: Best Distance = 535.37
Generation 237/1000: Best Distance = 480.11
Generation 279/1000: Best Distance = 478.34
{'best route': [0, 4, 6, 17, 1, 13, 5, 18, 2, 20, 7, 19, 3, 8, 12, 14, 9, 10, 15, 11, 16, 0]

<p>
    <img src="spinning-cat.gif" alt="Spinning Cat"> 
</p>
<h1>Loading...</h1>

<style>
    h1 {
        color: red;
        position: relative;
        animation: moveRightLeft 2s infinite ease-in-out; 
    }
    p {

    }
    img {
        width: 100px;
        height: 100px;
    }

    @keyframes moveRightLeft {
        0% {
            left: 0;
        }
        50% {
            left: 100px;
        }
        100% {
            left: 0;
        }
    }
</style>

In [ ]:
import plotly.express as px

loss_df = pd.DataFrame({'Generation': np.arange(0, len(best_distance_arr) * 50, 50), 'Best Distance': best_distance_arr})

plot = px.line(loss_df, x='Generation', y='Best Distance', title='Best Distance Over Generations')
plot.show()


In [ ]:
import plotly.graph_objects as go

def plot_route(coords_df, best_route):


    route_coords = coords_df.to_numpy()
    route_x = [route_coords[i][0] for i in best_route]
    route_y = [route_coords[i][1] for i in best_route]

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=route_x,
        y=route_y,
        mode='markers+lines',
        name='Route',
        marker=dict(size=10, color='blue'),
        line=dict(color='orange', width=2)
    ))

    fig.add_trace(go.Scatter(
        x=[route_x[0], route_x[-1]],  
        y=[route_y[0], route_y[-1]],  
        mode='markers',
        name='Depot',
        marker=dict(size=12, color='red')
    ))

    fig.update_layout(
        title='Best Route Plot',
        xaxis_title='X Coordinate',
        yaxis_title='Y Coordinate',
        showlegend=True
    )

    # Show plot
    fig.show()


In [ ]:
plot_route(coords_df, best_route)


# differntial evolution

In [ ]:
#Practical Advice: (e.g., start with NP = 10 * D, and CR = 0.9, F = 0.8)

num_customers = len(coords) - 1
num_populations = 10 * num_customers
num_populations



200

intialze population for differntial evolution

In [ ]:
diff_population=initialize_population(num_populations, num_customers)
diff_population

array([[ 0,  9,  8, ..., 19, 11,  0],
       [ 0, 20, 18, ..., 14, 11,  0],
       [ 0, 17, 15, ..., 20,  5,  0],
       ...,
       [ 0, 15,  8, ..., 19, 12,  0],
       [ 0,  9, 17, ..., 10, 15,  0],
       [ 0,  7, 10, ..., 18,  5,  0]], shape=(200, 22))

# mutation function for differntial 

In [ ]:

def mutate(diff_population, target_idx, F):

    pop_size = len(diff_population)

    indices = list(range(pop_size))
    indices.remove(target_idx) # so i dont chooce target for calculation
    x1, x2, x3 = diff_population[np.random.choice(indices, 3, replace=False)]

    mutant = x3 + F * (x2 - x1)


    mutant = np.round(mutant).astype(int)# formula can result in float values so i round them to int

    return mutant

F = 0.8  
target_idx = 0  
mutant_vector = mutate(diff_population, target_idx, F)
print(mutant_vector)


[ 0 27  9  6 18 22 16  9  9 -4  8  9  4 22  5 11 22  0  1  9  7  0]


In [ ]:
# mutant_vector = validate_and_repair(mutant_vector, num_customers)

In [ ]:
CR = 0.9  
target = diff_population[0]  
mutant = mutate(diff_population, 0, F=0.8)  

if np.random.rand() < CR:
    child = crossover(target, mutant)  
else:
    child = target  
child 

array([  0,   9,  19,  -6, -12,  16,  12,  17,  18,   3,   4,  20,  13,
        25,   1,  27,   8,  15,  -1,  -1,  -1,   0])